In [16]:
import pandas as pd
import math

# create ShapTime
def get_sub_set(Tn):
    
    mylist = list(range(Tn))
    sub_sets = [[]]
    for x in mylist:
        sub_sets.extend([item + [x] for item in sub_sets])
    return sub_sets 


def ValFunction(model, interp_x, Tn):
    
    dfx = supertime(interp_x)
    # Generate subsets of indexes
    subset = get_sub_set(Tn)   
    
    # Generate the baseline
    y_results = model.predict(interp_x)
    baseline = sum(y_results)/len(interp_x)
    
    val_results = []
    
    for i in range(1, len(subset)):
        x_i = dfx[subset[i][0]]
        
        if len(subset[i]) == 1:
            prediction = model.predict(x_i)
            results = (sum(prediction)/len(x_i)) - baseline
            val_results.append(results)
            
        else:
            for n in range(1, len(subset[i])):
                x_i = np.concatenate([x_i, dfx[subset[i][n]]], axis = 0)
            
            prediction = model.predict(x_i)
            results = (sum(prediction)/len(x_i)) - baseline
            val_results.append(results)
            
    val_results.insert(0,0.0)
            
    return subset, val_results


def index(Si, subset):
    for i in range(len(subset)):
        if Si == subset[i]:
            index = i
        else:
            pass
    return index


def ShapleyValues(model, interp_x, Tn):
    
    subset, val_results = ValFunction(model, interp_x, Tn)
    shapley_values = []
    for i in range (Tn):
        shapley = []
        for n in range(len(subset)):
            if i not in subset[n]:
                Si = subset[n]+[i]
                Si.sort()
                Si_num = index(Si, subset)
            
                S_num = len(subset[n])
                N = Tn
            
                weight = (math.factorial(S_num) * math.factorial((N-S_num-1))) / math.factorial(N)
                val = val_results[Si_num] - val_results[n]
                shapley_i = weight * val
            
                shapley.append(shapley_i)
            else:
                pass
            
        shapley_values.append(sum(shapley))
        del shapley
        
    return shapley_values

In [17]:
import numpy as np
from sklearn.linear_model import LinearRegression

# Generate sample data
np.random.seed(0)
X_train = np.random.rand(100, 10, 8)  # Input data: 100 samples, 10 time steps, 8 features
y_train = np.random.rand(100, 4, 8)   # Output data: 100 samples, 4 time steps, 8 features


# Define a black-box model that takes 3D input and produces 3D output
class ExampleModel:
    def predict(self, X):
        return np.random.rand(X.shape[0], 4, 8)
# Initialize the black-box model
model = ExampleModel()


# Define the number of time steps
Tn = 10

# Function to simulate supertime
def supertime(interp_x):
    dfx = []
    for i in range(interp_x.shape[1]):  # Iterate over the second axis (time steps)
        dfx.append(interp_x[:, i, :])  # Select all samples at the current time step
    return dfx



# Compute Shapley values
shapley_values = ShapleyValues(model, X_train, Tn)


# Print Shapley values for each time step
print("Shapley values for each time step:")
for i, shapley_value in enumerate(shapley_values):
    print(f"Time step {i+1}: {shapley_value}")


Shapley values for each time step:
Time step 1: [[-2.71563378e-03 -9.67365538e-04  1.29624848e-02 -5.39228363e-03
   4.96418171e-03 -4.89989192e-03 -1.45903771e-02 -1.80328239e-03]
 [ 3.91408133e-03 -2.72475193e-03  2.88413164e-03 -1.55050342e-03
   2.51221023e-03  3.15951254e-03  3.40966478e-03  2.54396347e-03]
 [ 2.34615759e-03  3.78188432e-03  1.86391219e-03 -9.80290463e-03
   1.16872917e-02 -6.89484202e-03 -2.09789490e-03 -3.90287245e-03]
 [-2.36641499e-03 -7.26922271e-04  2.50059202e-03  1.63412332e-03
   4.14464570e-03 -3.12541536e-03 -4.50040583e-05 -1.37131191e-03]]
Time step 2: [[-5.35084032e-05 -5.15243762e-03  7.30762494e-03  3.59459254e-03
   9.72397493e-03 -3.32050520e-03 -7.58557375e-03 -2.33392430e-03]
 [ 1.15056279e-02  1.15544161e-03  4.17861521e-04  4.49670953e-03
   1.62694384e-03  4.14239204e-03  7.05176699e-03 -3.60923063e-03]
 [ 4.55067022e-03  8.02245024e-03 -1.26170878e-03 -1.75571746e-03
   5.75336818e-03 -2.69810456e-03 -3.95257481e-04  3.93005168e-04]
 [-1.19

In [18]:
shapley_value.shape

(4, 8)

In [19]:
shapley_value[0]

array([-0.00466172, -0.00365354,  0.01147795, -0.00628028,  0.00346955,
       -0.00646036, -0.00532743, -0.00333071])

In [1]:
import pandas as pd
import numpy as np
import math
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Simulate weather data (wind speed, temperature, humidity, precipitation)
np.random.seed(0)
num_samples = 1000
num_features = 4
time_steps = 10  # Historical time steps
forecast_steps = 4  # Forecasting steps

wind_speed = np.random.rand(num_samples, time_steps)
temperature = np.random.rand(num_samples, time_steps)
humidity = np.random.rand(num_samples, time_steps)
precipitation = np.random.rand(num_samples, time_steps)

# Combine features into a DataFrame
weather_data = pd.DataFrame({
    'Wind Speed': wind_speed.ravel(),
    'Temperature': temperature.ravel(),
    'Humidity': humidity.ravel(),
    'Precipitation': precipitation.ravel()
})

# Simulate target (wind speed and temperature forecast)
target_wind_speed = np.random.rand(num_samples, forecast_steps)
target_temperature = np.random.rand(num_samples, forecast_steps)

# Combine targets into a DataFrame
forecast_data = pd.DataFrame({
    'Target Wind Speed': target_wind_speed.ravel(),
    'Target Temperature': target_temperature.ravel()
})




2024-03-28 02:01:22.668728: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-28 02:01:22.696141: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-28 02:01:22.696164: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-28 02:01:22.696872: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-28 02:01:22.701530: I tensorflow/core/platform/cpu_feature_guar

In [4]:
# Concatenate weather data and forecast data
full_data = pd.concat([weather_data, forecast_data], axis=1)
full_data

,Wind Speed,Temperature,Humidity,Precipitation,Target Wind Speed,Target Temperature
0,0.548814,0.748268,0.392173,0.758125,0.369256,0.726851
1,0.715189,0.180203,0.041157,0.503319,0.211326,0.906916
2,0.602763,0.389023,0.923301,0.177017,0.476905,0.383102
3,0.544883,0.037600,0.406235,0.832537,0.082234,0.221634
4,0.423655,0.011788,0.944282,0.516825,0.237659,0.046115
...,...,...,...,...,...,...
9995,0.550447,0.748175,0.622359,0.668383,NaN,NaN
9996,0.397151,0.298267,0.531140,0.994911,NaN,NaN
9997,0.758430,0.446456,0.852904,0.588224,NaN,NaN
9998,0.023787,0.360127,0.734230,0.424615,NaN,NaN


In [ ]:
# Prepare data for modeling
X = full_data.drop(['Target Wind Speed', 'Target Temperature'], axis=1).values.reshape(-1, time_steps, num_features)
y = full_data[['Target Wind Speed', 'Target Temperature']].values.reshape(-1, forecast_steps, 2)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:


# Define the Shapley value computation functions

def get_sub_set(Tn):
    mylist = list(range(Tn))
    sub_sets = [[]]
    for x in mylist:
        sub_sets.extend([item + [x] for item in sub_sets])
    return sub_sets 

def supertime(interp_x):
    dfx = []
    for i in range(interp_x.shape[1]):  # Iterate over the second axis (time steps)
        dfx.append(interp_x[:, i, :])  # Select all samples at the current time step
    return dfx

def ValFunction(model, interp_x, Tn):
    dfx = supertime(interp_x)
    # Generate subsets of indexes
    subset = get_sub_set(Tn)   
    
    # Generate the baseline
    y_results = model.predict(interp_x)
    baseline = np.mean(y_results, axis=0)
    
    val_results = []
    
    for i in range(1, len(subset)):
        x_i = dfx[subset[i][0]]
        
        if len(subset[i]) == 1:
            prediction = model.predict(x_i)
            results = (np.mean(prediction, axis=0)) - baseline
            val_results.append(results)
            
        else:
            for n in range(1, len(subset[i])):
                x_i = np.concatenate([x_i, dfx[subset[i][n]]], axis=0)
            
            prediction = model.predict(x_i)
            results = (np.mean(prediction, axis=0)) - baseline
            val_results.append(results)
            
    val_results.insert(0, np.zeros(baseline.shape))
            
    return subset, val_results

def index(Si, subset):
    for i in range(len(subset)):
        if Si == subset[i]:
            return i
    return -1

def ShapleyValues(model, interp_x, Tn):
    subset, val_results = ValFunction(model, interp_x, Tn)
    shapley_values = []
    for i in range(Tn):
        shapley = []
        for n in range(len(subset)):
            if i not in subset[n]:
                Si = subset[n]+[i]
                Si.sort()
                Si_num = index(Si, subset)
                S_num = len(subset[n])
                N = Tn
                weight = (math.factorial(S_num) * math.factorial((N-S_num-1))) / math.factorial(N)
                val = val_results[Si_num] - val_results[n]
                shapley_i = weight * val
                shapley.append(shapley_i)
        shapley_values.append(np.sum(shapley, axis=0))
    return shapley_values

# Create a simple LSTM model for forecasting
def create_lstm_model(input_shape, output_shape):
    model = Sequential([
        LSTM(64, input_shape=input_shape, return_sequences=True),
        LSTM(64),
        Dense(output_shape)
    ])
    return model

# Normalize features
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train.reshape(-1, num_features)).reshape(X_train.shape)
X_test_scaled = scaler.transform(X_test.reshape(-1, num_features)).reshape(X_test.shape)

# Define input shape and output shape for the LSTM model
input_shape = (time_steps, num_features)
output_shape = forecast_steps * 2  # 2 targets: wind speed and temperature

# Create and compile the LSTM model
model = create_lstm_model(input_shape, output_shape)
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, validation_data=(X_test_scaled, y_test))

# Compute Shapley values
shapley_values = ShapleyValues(model, X_test_scaled, time_steps)

# Print Shapley values for each time step
print("Shapley values for each time step:")
for i, shapley_val in enumerate(shapley_values):
    print(f"Time Step {i + 1}: {shapley_val}")

In [ ]:
# Define the Shapley value computation functions

def get_sub_set(Tn):
    mylist = list(range(Tn))
    sub_sets = [[]]
    for x in mylist:
        sub_sets.extend([item + [x] for item in sub_sets])
    return sub_sets 

def supertime(interp_x):
    dfx = []
    for i in range(interp_x.shape[1]):  # Iterate over the second axis (time steps)
        dfx.append(interp_x[:, i, :])  # Select all samples at the current time step
    return dfx

def ValFunction(model, interp_x, Tn):
    dfx = supertime(interp_x)
    # Generate subsets of indexes
    subset = get_sub_set(Tn)   
    
    # Generate the baseline
    y_results = model.predict(interp_x)
    baseline = np.mean(y_results, axis=0)
    
    val_results = []
    
    for i in range(1, len(subset)):
        x_i = dfx[subset[i][0]]
        
        if len(subset[i]) == 1:
            print(x_i)
            myBlackbox = BlackBoxWrapper(model) ## Ajout
            #prediction = model.predict(x_i)
            prediction = myBlackbox.predict(x_i)
            print(prediction)
            results = (np.mean(prediction, axis=0)) - baseline
            val_results.append(results)
            
        else:
            for n in range(1, len(subset[i])):
                x_i = np.concatenate([x_i, dfx[subset[i][n]]], axis=0)
            
            #prediction = model.predict(x_i)
            prediction = myBlackbox.predict(x_i)
            results = (np.mean(prediction, axis=0)) - baseline
            val_results.append(results)
            
    val_results.insert(0, np.zeros(baseline.shape))
            
    return subset, val_results

def index(Si, subset):
    for i in range(len(subset)):
        if Si == subset[i]:
            return i
    return -1

def ShapleyValues(model, interp_x, Tn):
    subset, val_results = ValFunction(model, interp_x, Tn)
    shapley_values = []
    for i in range(Tn):
        shapley = []
        for n in range(len(subset)):
            if i not in subset[n]:
                Si = subset[n]+[i]
                Si.sort()
                Si_num = index(Si, subset)
                S_num = len(subset[n])
                N = Tn
                weight = (math.factorial(S_num) * math.factorial((N-S_num-1))) / math.factorial(N)
                val = val_results[Si_num] - val_results[n]
                shapley_i = weight * val
                shapley.append(shapley_i)
        shapley_values.append(np.sum(shapley, axis=0))
    return shapley_values